In [9]:
!pip install --upgrade transformers
!pip install rouge_Score
!pip install transformers datasets torch


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
from rouge_score import rouge_scorer
from datasets import load_dataset
from datasets import Dataset

load dataset and prepare 20% of it to test /the model performance

In [15]:
dataset = pd.read_csv ("/content/drive/MyDrive/newMind_task/conclusions.csv")
length = len(dataset)
print( length)
portion = length * 0.2
test_dataset = dataset['text'].head(int(portion))
print(test_dataset.shape)

3351
(670,)


In [16]:
save_directory = "/content/drive/MyDrive/newMind_task/T5_llm_model"
test_summaries = []
test_texts = []
# Load pre-trained T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
def gernerate_summary( text ):
  inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
  summary_ids = model.generate(inputs,
                              max_length=150,
                              min_length=30,
                              length_penalty=2.0,
                              num_beams=4,
                              early_stopping=True)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return text , summary

for text in test_dataset:
  text,summery = gernerate_summary(text)
  test_summaries.append(summery)
  test_texts.append(text)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [17]:

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize accumulators for the metrics
total_rouge1 = {'precision': 0, 'recall': 0, 'fmeasure': 0}
total_rouge2 = {'precision': 0, 'recall': 0, 'fmeasure': 0}
total_rougeL = {'precision': 0, 'recall': 0, 'fmeasure': 0}

# Loop through the dataset and calculate scores
for text, summary in zip(test_texts, test_summaries):
    scores = scorer.score(text, summary)
    total_rouge1['precision'] += scores['rouge1'].precision
    total_rouge1['recall'] += scores['rouge1'].recall
    total_rouge1['fmeasure'] += scores['rouge1'].fmeasure

    total_rouge2['precision'] += scores['rouge2'].precision
    total_rouge2['recall'] += scores['rouge2'].recall
    total_rouge2['fmeasure'] += scores['rouge2'].fmeasure

    total_rougeL['precision'] += scores['rougeL'].precision
    total_rougeL['recall'] += scores['rougeL'].recall
    total_rougeL['fmeasure'] += scores['rougeL'].fmeasure

# Calculate the average for each metric
num_samples = len(test_texts)
avg_rouge1 = {key: value / num_samples for key, value in total_rouge1.items()}
avg_rouge2 = {key: value / num_samples for key, value in total_rouge2.items()}
avg_rougeL = {key: value / num_samples for key, value in total_rougeL.items()}

# Create a DataFrame to display the average results
data = {
    "Metric": ["ROUGE-1", "ROUGE-2", "ROUGE-L"],
    "Average Precision": [avg_rouge1['precision'], avg_rouge2['precision'], avg_rougeL['precision']],
    "Average Recall": [avg_rouge1['recall'], avg_rouge2['recall'], avg_rougeL['recall']],
    "Average F1-Score": [avg_rouge1['fmeasure'], avg_rouge2['fmeasure'], avg_rougeL['fmeasure']]
}

df_avg = pd.DataFrame(data)

# Display the DataFrame
df_avg


,Metric,Average Precision,Average Recall,Average F1-Score
0,ROUGE-1,0.908175,0.592551,0.670775
1,ROUGE-2,0.856023,0.559733,0.630528
2,ROUGE-L,0.880181,0.576542,0.651510


In [19]:
# Save the model
model.save_pretrained(save_directory)
# Save the tokenizer
tokenizer.save_pretrained(save_directory)

('/content/drive/MyDrive/newMind_task/T5_llm_model/tokenizer_config.json',
 '/content/drive/MyDrive/newMind_task/T5_llm_model/special_tokens_map.json',
 '/content/drive/MyDrive/newMind_task/T5_llm_model/spiece.model',
 '/content/drive/MyDrive/newMind_task/T5_llm_model/added_tokens.json')

In [21]:
model = T5ForConditionalGeneration.from_pretrained(save_directory)
tokenizer = T5Tokenizer.from_pretrained(save_directory)
def use_llm_model ( input_text):

  inputs = tokenizer("summarize: " + input_text, return_tensors="pt", max_length=512, truncation=True)
  summary_ids = model.generate(inputs.input_ids, max_length=150, num_beams=4, early_stopping=True)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return summary

summary = use_llm_model("Though people were not satified about how the landform was a natural landform, in all, we new that alieans did not form the face. I would like to know how the landform was formed. we know now that life on Mars doesn't exist.")

summary

"alieans were not satified about how the landform was a natural landform. we know that life on Mars doesn't exist."